In [1]:
from openmdao.api import IndepVarComp, Component, Problem, Group

import json
from fusedwind.core.FUSEDobj import FUSEDobj, flatten
from fusedwind.core.FUSEDComponent import FUSEDWrapper, FUSEDComponent
import numpy as np
from functools import reduce

In [2]:
{1,2,3}.union({1,3})

{1, 2, 3}

#### You can define the component as a class

In [3]:
class wake_model(FUSEDComponent):
    def __init__(self, wt1, wt2, wind_speed, wind_direction):
        """        
        Parameters
        ----------
        wt1: FUSEDobj
            turbine 1
        wt2: FUSEDobj
            turbine 2 in the wake of turbine 1
        """
        super(wake_model, self).__init__()
        self.wt1 = wt1
        self.wt2 = wt2
        self.ws = wind_speed
        self.wd = wind_direction
        self.D1 = wt1.rotor_diameter
        self.D2 = wt2.rotor_diameter
        
        # run _execute and automatically figure out what are the params and
        # unknowns of the Component. 
        # Note: Obviously, don't do that is `_execute()` takes ages!
        self.auto_configure() 
        
    def _execute(self):
        self.wt2.power = self.ws**3.0 * (1.0 - self.wt1.c_t)
        self.wt2.c_p = self.wt2.power/(np.pi * (self.D1/2.0)**2.0 * 1.225 * self.wd**3.0)
        self.D2 = self.D1 / 2.0

#### Or defined as a pure python function

In [24]:
def wake_func(ws, wd, wt1, wt2):
        wt2.power = ws**3.0 * (1.0 - wt1.c_t)
        wt2.c_p = wt2.power/(np.pi * (wt1.rotor_diameter/2.0)**2.0 * 1.225 * wd**3.0)
        wt2.D2 = wt1.rotor_diameter / 2.0
        return wt2

In [25]:
site = FUSEDobj({'layout':[
    {
        'name': 'wt1',
        'power': 2000.0,
        'c_t': 0.8,
        'c_p': 0.5,
        'rotor_diameter': 80.0,
    },
   {
        'name': 'wt2',
        'power': 2000.0,
        'c_t': 0.8,
        'c_p': 0.5,                
        'rotor_diameter': 80.0,
    },
    {
        'name': 'wt3',
        'power': 2000.0,
        'c_t': 0.8,
        'c_p': 0.5,                
        'rotor_diameter': 80.0,
    }
        ]})

inflow = FUSEDobj({
    'wind_speed':8.0,
    'wind_direction': 270.0
    })

wind_speed = FUSEDobj(8.0, 'wind_speed')
wind_direction = FUSEDobj(210.0, 'wind_direction')
#wind_speed = FUSEDobj(8.0) #TODO: fix the missing name bug

edge case wind_speed 8.0
edge case wind_direction 210.0


In [26]:
wind_speed

8.0

In [27]:
site

{
  "layout": [
    {
      "rotor_diameter": 80.0,
      "power": 2000.0,
      "c_t": 0.8,
      "name": "wt1",
      "c_p": 0.5
    },
    {
      "rotor_diameter": 80.0,
      "power": 2000.0,
      "c_t": 0.8,
      "name": "wt2",
      "c_p": 0.5
    },
    {
      "rotor_diameter": 80.0,
      "power": 2000.0,
      "c_t": 0.8,
      "name": "wt3",
      "c_p": 0.5
    }
  ]
}

In [28]:
wm = wake_model(site.layout.wt1, site.layout.wt2, wind_speed, inflow.wind_direction)

input: wind_speed 8.0
input: layout:wt1:rotor_diameter 80.0
input: wind_direction 270.0
input: layout:wt1:c_t 0.8
output: layout:wt2:rotor_diameter 40.0
output: layout:wt2:c_p 8.448949612900222e-10
output: layout:wt2:power 102.39999999999998


#### You can also use a pure function and wrap it by indicating which inputs and output objects are used

In [30]:
wm2 = FUSEDWrapper(wake_func, 
 inputs=[
    wind_speed,
    wind_direction, 
    site.layout.wt1,
    site.layout.wt2], 
 outputs=[site.layout.wt2])

input: wind_speed 8.0
input: layout:wt1:rotor_diameter 80.0
input: wind_direction 210.0
input: layout:wt1:c_t 0.8
output: layout:wt2:c_p 1.795709699068298e-09
output: layout:wt2:power 102.39999999999998


#### You can create a project by using the address of the FUSEDobjects

In [31]:
wind_speed

8.0

In [32]:
 top = Problem()

root = top.root = Group()

root.add('p1', wind_speed.IndepVarComp(), ['*'])
# You can also do this:
root.add('p2', IndepVarComp(wind_direction.address, 220.0), ['*'])

root.add('wm', wm,['*'])
top.setup()
top.run()

##############################################
Setup: Checking for potential issues...

No recorders have been specified, so no data will be saved.

The following parameters have no associated unknowns:
layout:wt1:c_t
layout:wt1:rotor_diameter

Setup: Check complete.
##############################################



In [33]:
 top = Problem()

root = top.root = Group()

root.add('p1', wind_speed.IndepVarComp(), ['*'])
# You can also do this:
root.add('p2', IndepVarComp(wind_direction.address, 220.0), ['*'])

root.add('wm2', wm2,['*'])
top.setup()
top.run()

##############################################
Setup: Checking for potential issues...

No recorders have been specified, so no data will be saved.

The following parameters have no associated unknowns:
layout:wt1:c_t
layout:wt1:rotor_diameter

Setup: Check complete.
##############################################



#### The FUSEDobject used in the OpenMDAO have been updated under the hood (yay!)

In [34]:
site

{
  "layout": [
    {
      "rotor_diameter": 80.0,
      "power": 2000.0,
      "c_t": 0.8,
      "name": "wt1",
      "c_p": 0.5
    },
    {
      "rotor_diameter": 40.0,
      "power": 102.39999999999998,
      "c_t": 0.8,
      "name": "wt2",
      "c_p": 1.561801983759533e-09
    },
    {
      "rotor_diameter": 80.0,
      "power": 2000.0,
      "c_t": 0.8,
      "name": "wt3",
      "c_p": 0.5
    }
  ]
}

#### Some weird bug happens if you try to rerun again:

In [35]:
wind_speed.value.__class__.__name__

'float64'

In [36]:
wind_direction.value.__class__.__name__

'float64'

In [37]:
top = Problem()

root = top.root = Group()

root.add('p1', wind_speed.IndepVarComp(), ['*'])
# You can also do this:
root.add('p2', IndepVarComp(wind_direction.address, 220.0), ['*'])

root.add('wm2', wm2,['*'])
top.setup()
top.run()

TypeError: Type <class 'numpy.float64'> of source 'p1.wind_speed' (wind_speed) must be the same as type <class 'float'> of target 'wm2.wind_speed' (wind_speed).

In [38]:
top.root.wm2._params_dict['wm2.wind_speed']['val'].__class__

float